In [1]:
pip install scipy --upgrade

Note: you may need to restart the kernel to use updated packages.


In [2]:
from scipy.io import arff
import pandas as pd

In [3]:
file_path = "C:\\dataset.arff"

In [4]:
import os
file_path = "C:\\dataset.arff"
if os.path.exists(file_path):
    print("File exists")
else:
    print("File not found")

File exists


In [5]:
!pip install liac-arff
import arff
with open(file_path, 'r') as f:
    dataset = arff.load(f)

In [6]:
try:
    data, meta = arff.loadarff(file_path)
except ValueError as e:
    print(f"Error while parsing header, error was: {e}")

AttributeError: module 'arff' has no attribute 'loadarff'

In [7]:
data, meta = arff.loadarff(file_path)

AttributeError: module 'arff' has no attribute 'loadarff'

In [11]:
from scipy.io import arff
import pandas as pd

file_path = "C:\Users\makar\Desktop\Springboard\Capstone project 2\From Lucas\dataset.arff"

# Load the ARFF file
data, meta = arff.loadarff(file_path)

# Convert to a pandas DataFrame
df = pd.DataFrame(data)

# Display the DataFrame
df.head()

SyntaxError: (unicode error) 'unicodeescape' codec can't decode bytes in position 2-3: truncated \UXXXXXXXX escape (2523961549.py, line 4)

In [9]:
with open(file_path, 'r') as file:
    arff_content = file.read()

# Replace problematic quotation marks in the attribute names
fixed_arff_content = arff_content.replace('"', '')

# Save the fixed ARFF content to a new file
fixed_file_path = "C:\\fixed_dataset.arff"
with open(fixed_file_path, 'w') as file:
    file.write(fixed_arff_content)

# Load the fixed ARFF file
data, meta = arff.loadarff(fixed_file_path)

# Convert to a pandas DataFrame
df = pd.DataFrame(data)

# Display the DataFrame
df.head()

PermissionError: [Errno 13] Permission denied: 'C:\\fixed_dataset.arff'

In [10]:
with open(file_path, 'r') as file:
    arff_content = file.read()

# Replace problematic quotation marks in the attribute names
fixed_arff_content = arff_content.replace('"', '')

# Save the fixed ARFF content to a new file
fixed_file_path = "C:\\fixed_dataset.arff"
with open(fixed_file_path, 'w') as file:
    file.write(fixed_arff_content)

# Load the fixed ARFF file
data, meta = arff.loadarff(fixed_file_path)

# Convert to a pandas DataFrame
df = pd.DataFrame(data)

# Display the DataFrame
df.head()

PermissionError: [Errno 13] Permission denied: 'C:\\fixed_dataset.arff'